<a href="https://colab.research.google.com/github/Fuzz117/MLBLR/blob/Fuzz117%2FMLBLR%2Fnew/session61.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import re
import string
from google.colab import drive
drive.mount("/content/gdrive")



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
filename = "/content/gdrive/My Drive/Colab Notebooks/session6/text.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
raw_text = re.sub('['+string.punctuation+']', '', raw_text)

In [0]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [0]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: "), n_chars
print ("Total Vocab: "), n_vocab
print(chars)

Total Characters: 
Total Vocab: 
['\n', ' ', '0', '3', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [0]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: "), n_patterns

Total Patterns: 


(None, 136016)

In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
#model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_2 (Dense)              (None, 30)                7710      
Total params: 797,214
Trainable params: 797,214
Non-trainable params: 0
_________________________________________________________________


In [0]:
filepath="/content/gdrive/My Drive/Colab Notebooks/session6/weights6.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


In [0]:
# fit the model
model.fit(X, y, epochs=100, batch_size=512, callbacks=callbacks_list)


Epoch 1/100
136016/136016 [==============================] - 74s 543us/step - loss: 2.8883 - acc: 0.1816

Epoch 00001: loss improved from inf to 2.88835, saving model to /content/gdrive/My Drive/Colab Notebooks/session6/weights6.hdf5
Epoch 2/100
136016/136016 [==============================] - 72s 531us/step - loss: 2.7921 - acc: 0.2105

Epoch 00002: loss improved from 2.88835 to 2.79212, saving model to /content/gdrive/My Drive/Colab Notebooks/session6/weights6.hdf5
Epoch 3/100
136016/136016 [==============================] - 72s 527us/step - loss: 2.5705 - acc: 0.2697

Epoch 00003: loss improved from 2.79212 to 2.57049, saving model to /content/gdrive/My Drive/Colab Notebooks/session6/weights6.hdf5
Epoch 4/100
136016/136016 [==============================] - 72s 533us/step - loss: 2.3934 - acc: 0.3093

Epoch 00004: loss improved from 2.57049 to 2.39340, saving model to /content/gdrive/My Drive/Colab Notebooks/session6/weights6.hdf5
Epoch 5/100
136016/136016 [=========================

In [0]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	print(result,end='')
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")